# Jupyter notebook to create the evaluation dataset

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, decimate
from obspy import read
from tqdm import tqdm

In [2]:
path_data = '/home/data' 
noise_path = path_data + '/NoiseSegs'
whale_call_path = path_data + '/WhaleCalls'
true_negative_path = path_data + '/SeisSegs'

ls_noise = os.listdir(noise_path)
ls_whale_call = os.listdir(whale_call_path)
ls_true_negative = os.listdir(true_negative_path)

In [3]:
def bandfilt(tr, lower_cutoff=10, upper_cutoff=45, filter_order = 4):
    ''' Filter the trace using a bandpass filter.
    '''
    sr = tr.stats.sampling_rate
    tr_filt = tr
    nyquist_frequency = sr / 2
    b, a = butter(filter_order, [lower_cutoff / nyquist_frequency, upper_cutoff / nyquist_frequency], btype='band')
    tr_filt_data = filtfilt(b, a, tr.data)
    tr_filt = tr.copy()
    tr_filt.data = tr_filt_data

    return tr_filt

In [4]:
def resample(tr, target_sr = 100):
    ''' Resample the trace to a target sampling rate.
    '''
    factor = round(tr.stats.sampling_rate / target_sr)
    tr.decimate(factor=factor, strict_length=False, no_filter=True)

    return tr, factor

In [7]:
def homogenizer(tr, target_sr=100, target_filt = [10, 45], print_details=False):
    '''Receives a trace and resample it in case sampling rate is different from target sampling rate, also apply a filter to the trace.
    '''
    sr = tr.stats.sampling_rate
    if sr > target_sr + 1 or sr < target_sr - 1: # if sampling rate is not target_sr
        tr, factor = resample(tr, target_sr=target_sr) # antialiasing filter
        if print_details:
            print(f'Resampled from {sr} Hz to {target_sr} Hz.')
            print(f'Decimation factor: {factor}')
            print(f'New sampling rate: {tr.stats.sampling_rate}')
            print('\n')
    tr = bandfilt(tr, lower_cutoff=target_filt[0], upper_cutoff=target_filt[1]) # antialiasing filter
    tr.data /= max(tr.data)

    return tr


In [8]:
save_path = path_data + '/eval_dataset'

In [9]:
n_call_true_noise = 1000 # whales + true noise of different amplitudes (no need to include noises that are too high)
n_call_superimposed_call = 1000 # whales + superimposed calls + true noise of different amplitudes
n_true_noise = 2000 # true noises (with seismic shots)

# Whale call + True noises
type_ = 1
for i in tqdm(range(n_call_true_noise)):
    len_call = 1
    len_true_noise = 0
    while len_call > len_true_noise:
        fname = ls_whale_call[np.random.randint(0, len(ls_whale_call) - 1)]
        call = read(whale_call_path + '/' + fname)
        call = homogenizer(call[0], target_sr=100, target_filt = [10, 45], print_details=False)
        fname2 = ls_noise[np.random.randint(0, len(ls_noise) - 1)]
        true_noise = read(noise_path + '/' + fname2)
        true_noise = homogenizer(true_noise[0], target_sr=100, target_filt = [10, 45], print_details=False)
        len_call = len(call.data)
        len_true_noise = len(true_noise.data)
    # Calculate noie level based on Gaussian distribution
    mean = 0.5
    std_dev = 0.2
    noise_level = np.clip(np.random.normal(mean, std_dev), 0.1, 0.9)
    # Random index to join the two traces
    dif = len_true_noise - len_call
    idx_rand = np.random.randint(0, dif)
    if np.random.randint(0, 2) == 0:
        sign = -1
    else:
        sign = 1
    if sign > 0:
        true_noise_v2 = true_noise.data[idx_rand:idx_rand + len(call.data)]
    else:
        true_noise_v2 = true_noise.data[-idx_rand-len(call.data):-idx_rand ]
    true_noise_v2 *= noise_level
    #call.plot();
    call.data += true_noise_v2
    #call.plot();
    call.write(save_path + '/' + f'{type_}_{i}_' + fname)
    #break

# Whale call + Superimposed calls + true noise of different amplitudes
type_ = 2
for i in tqdm(range(n_call_superimposed_call)):
    len_call = 1
    len_call2 = 0
    while len_call > len_call2:
        fname = ls_whale_call[np.random.randint(0, len(ls_whale_call) - 1)]
        call = read(whale_call_path + '/' + fname)
        call = homogenizer(call[0], target_sr=100, target_filt = [10, 45], print_details=False)
        fname2 = ls_whale_call[np.random.randint(0, len(ls_whale_call) - 1)]
        call2 = read(whale_call_path + '/' + fname2)
        call2 = homogenizer(call2[0], target_sr=100, target_filt = [10, 45], print_details=False)
        len_call = len(call.data)
        len_call2 = len(call2.data)
    # Calculate noise level based on Gaussian distribution
    mean = 1
    std_dev = 1
    noise_level = np.clip(np.random.normal(mean, std_dev), 0.3, 2)
    # Random index to join the two traces
    dif = len_call2 - len_call
    if dif != 0:
        idx_rand = np.random.randint(0, dif)
    else:
        idx_rand = 0
    call2_v2 = call2.data[idx_rand:idx_rand + len(call.data)]
    delay = idx_rand * call2.stats.delta # in seconds
    pick_corrected = call2.stats.sac.a - delay
    call2_v2 *= noise_level
    #call.plot(color='red');
    call.data += call2_v2
    #call.plot(color='red');
    call.stats['sac']['t0'] = pick_corrected
    call.stats['sac']['kt0'] = 'P'
    call.write(save_path + '/' + f'{type_}_{i}_' + fname)
    #break

# True noises
type_ = 0
for i in tqdm(range(n_true_noise)):
    fname = ls_true_negative[np.random.randint(0, len(ls_true_negative) - 1)]
    noise = read(true_negative_path + '/' + fname)
    noise = homogenizer(noise[0], target_sr=100, target_filt = [10, 45], print_details=False)
    #noise.plot(color='blue');
    noise.write(save_path + '/' + f'{type_}_{i}_' + fname)
    #break

100%|███████████████████████████████████████| 2000/2000 [00:29<00:00, 67.23it/s]


type_ == 0: True noises (with seismic shots)

type_ == 1: Whale call + True noise (without seismic shots)

type_ == 2: Whale call + Superimposed calls + true noise of different amplitudes
